In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Optical Flow

In [2]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [3]:
# Small windows more sensitive to smaller motions (se debe ajustar)
# max level: realiza un subsample, bajando la resolucion de la imagen y analiza, en este caso 2 level
# criteria: usa un epsilon o una cantidad de iteraciones
lk_params = dict(winSize = (200,200),maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS  | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [4]:
cap = cv2.VideoCapture(0)

# Parametros de inicializacion del algoritmo
# previous frame
ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# PTS TO TRACK

prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    
    # current frame
    ret , frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # la magia
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    
    # status es igual a 1 , cuando el flujo de las caractersiticas (flow features) ha sido encontrado, en tal 
    #caso entonces refresca estas variables. Donde estaban los puntos antes y donde estan ahora, es eso basicamente.
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    for i, (new,prev) in enumerate(zip(good_new, good_prev)):
        
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
            
        mask = cv2.line(mask,(int(x_new),int(y_new)),(int(x_prev),int(y_prev)),(0,255,0),3)
        
        frame = cv2.circle(frame,(int(x_new),int(y_new)),8,(0,0,255),-1)
        
    img = cv2.add(frame, mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    
    if k == 27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()
cap.release()